In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\adsb_sample.csv")

print("Original Shape:", df.shape)
df.head()

Original Shape: (1526689, 16)


,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact
0,1496620800,4bccb9,52.084916,11.445394,175.320557,305.321045,NaN,SXS2WY,False,False,False,1000.0,7459.98,7581.90,1.496621e+09,1.496621e+09
1,1496620800,502cb2,49.637375,2.871841,232.936358,305.199350,-0.32512,MON55BR,False,False,False,2770.0,10988.04,11033.76,1.496621e+09,1.496621e+09
2,1496620800,4bccaf,49.850464,12.522783,196.684570,288.764648,NaN,SXS7R,False,False,False,3215.0,11582.40,11772.90,1.496621e+09,1.496621e+09
3,1496620800,4008e1,50.780847,9.262612,214.205322,292.747192,NaN,TCX229,False,False,False,3462.0,10363.20,NaN,1.496621e+09,1.496621e+09
4,1496620800,4070e3,50.722992,3.823700,221.822622,297.634765,0.32512,EXS14D,False,False,False,7775.0,10972.80,11033.76,1.496621e+09,1.496621e+09


In [ ]:
import pandas as pd
import numpy as np

def clean_icao(x):
    if pd.isna(x):
        return None
    
    x = str(x).strip()

 
    if "E" in x.upper():
        try:
            return format(int(float(x)), "x")  
        except:
            return x

    try:
        f = float(x)
        if f.is_integer():
            return format(int(f), "x") 
    except:
        pass

    if all(c in "0123456789abcdefABCDEF" for c in x):
        return x.lower()

    return x.lower()


clean = df.copy()
clean["icao24"] = clean["icao24"].apply(clean_icao)

clean.head()



,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact
0,1496620800,4bccb9,52.084916,11.445394,175.320557,305.321045,NaN,SXS2WY,False,False,False,1000.0,7459.98,7581.90,1.496621e+09,1.496621e+09
1,1496620800,502cb2,49.637375,2.871841,232.936358,305.199350,-0.32512,MON55BR,False,False,False,2770.0,10988.04,11033.76,1.496621e+09,1.496621e+09
2,1496620800,4bccaf,49.850464,12.522783,196.684570,288.764648,NaN,SXS7R,False,False,False,3215.0,11582.40,11772.90,1.496621e+09,1.496621e+09
3,1496620800,9c90,50.780847,9.262612,214.205322,292.747192,NaN,TCX229,False,False,False,3462.0,10363.20,NaN,1.496621e+09,1.496621e+09
4,1496620800,3e1a70,50.722992,3.823700,221.822622,297.634765,0.32512,EXS14D,False,False,False,7775.0,10972.80,11033.76,1.496621e+09,1.496621e+09


In [5]:
clean.to_csv(
    r"C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\cleaned_adsb.csv",
    index=False
)

print("Saved cleaned file.")


Saved cleaned file.


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\adsb_sample.csv")

cleaned = df.copy()


cleaned["icao24"] = cleaned["icao24"].astype(str).str.replace(r"\.0$", "", regex=True)

def fix_icao(x):
    x = x.strip()
    if "E" in x.upper():
        try:
            return hex(int(float(x)))[2:]
        except:
            return x
    return x

cleaned["icao24"] = cleaned["icao24"].apply(fix_icao)

bool_cols = ["onground","alert","spi"]
for col in bool_cols:
    if col in cleaned.columns:
        cleaned[col] = cleaned[col].astype(str).str.upper().map({"TRUE":1, "FALSE":0})

cleaned = cleaned.replace([np.inf, -np.inf], np.nan)


fill_cols = [
    "velocity","vertrate","baroaltitude","geoaltitude",
    "lat","lon","heading"
]

cleaned[fill_cols] = cleaned.groupby("icao24")[fill_cols].transform(
    lambda g: g.ffill().bfill()
)

out_path = r"C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\cleaned_adsb.csv"
cleaned.to_csv(out_path, index=False)

print("CLEANED SHAPE:", cleaned.shape)
print("Saved to:", out_path)


CLEANED SHAPE: (1526689, 16)
Saved to: C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\cleaned_adsb.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

df = pd.read_csv(r"C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\cleaned_adsb.csv")

print("Loaded:", df.shape)


features = [
    "velocity",
    "vertrate",
    "baroaltitude",
    "geoaltitude",
    "lat",
    "lon"
]

df_clust = df[features].copy()
df_clust = df_clust.replace([np.inf, -np.inf], np.nan)
df_clust = df_clust.dropna()

print("Rows usable for clustering:", df_clust.shape)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clust)

k = 6   
kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
labels = kmeans.fit_predict(X_scaled)

df.loc[df_clust.index, "phase"] = labels

out_path = r"C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\clustered_adsb.csv"
df.to_csv(out_path, index=False)

print("Clustering done! Saved to:", out_path)
df["phase"].value_counts(dropna=False)


Loaded: (1526689, 16)
Rows usable for clustering: (1058077, 6)
Clustering done! Saved to: C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\clustered_adsb.csv


phase
NaN    468612
0.0    416989
1.0    236163
5.0    153336
4.0    149950
2.0    101145
3.0       494
Name: count, dtype: int64

In [17]:
import plotly.express as px

phase_counts = df["phase"].value_counts().reset_index()
phase_counts.columns = ["phase_label", "count"]

fig = px.bar(
    phase_counts,
    x="phase_label",
    y="count",
    labels={"phase_label": "Cluster", "count": "Count"},
    title="Cluster Distribution"
)
fig.show()


In [18]:
fig = px.scatter(
    df.sample(40000),
    x="velocity",
    y="baroaltitude",
    color="phase",
    opacity=0.6,
    title="Velocity vs Altitude (Cluster-colored)"
)
fig.show()


In [22]:
import pandas as pd
import plotly.express as px

# Convert UNIX timestamp → datetime (if not already done)
df["time"] = pd.to_datetime(df["time"], unit="s", errors="coerce")

# Create cleaner, readable scatter plot
fig = px.scatter(
    df.sample(8000).sort_values("time"),    # reduced for clarity
    x="time",
    y="velocity",
    color="phase",
    opacity=0.5,
    title="Velocity Over Time — Cluster-Based Segmentation",
)

# Better marker size + layout
fig.update_traces(marker=dict(size=4))
fig.update_layout(
    xaxis_title="Time",
    yaxis_title="Velocity (m/s)",
    legend_title="Cluster (Phase)",
)

fig.show()


In [ ]:


import os
import math
import pickle
from pathlib import Path
from typing import Optional

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest


try:
    import tensorflow as tf
    from tensorflow.keras import layers, models, callbacks
    TF_AVAILABLE = True
except Exception:
    TF_AVAILABLE = False


try:
    import plotly.express as px
    PLOTLY_AVAILABLE = True
except Exception:
    PLOTLY_AVAILABLE = Fa
CSV_PATH = r"C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\adsb_sample.csv"
OUT_DIR = Path.cwd() / "pipeline_artifacts"
OUT_DIR.mkdir(parents=True, exist_ok=True)

CLEANED_CSV = OUT_DIR / "cleaned_features.csv"
SCALER_FILE = OUT_DIR / "scaler.pkl"
PCA_FILE = OUT_DIR / "pca.pkl"
IF_FILE = OUT_DIR / "isolation_forest.pkl"
AE_FILE = OUT_DIR / "autoencoder.h5"
AE_HISTORY = OUT_DIR / "ae_history.pkl"

def safe_read_csv(path):
    for sep in [",", "\t", ";"]:
        try:
            df = pd.read_csv(path, sep=sep, low_memory=False)
            if "time" in df.columns and "icao24" in df.columns:
                return df
        except Exception:
            continue
    return pd.read_csv(path, low_memory=False)

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1 = np.deg2rad(lat1)
    phi2 = np.deg2rad(lat2)
    dphi = phi2 - phi1
    dlambda = np.deg2rad(lon2 - lon1)
    a = np.sin(dphi/2.0)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2.0)**2
    return R * 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
print("Loading CSV:", CSV_PATH)
df = safe_read_csv(CSV_PATH)
print("Raw shape:", df.shape)

df.columns = [c.strip() for c in df.columns]
num_cols = ["time","lat","lon","velocity","heading","vertrate","baroaltitude","geoaltitude","lastposupdate","lastcontact"]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

if "icao24" in df.columns:
    df["icao24"] = df["icao24"].astype(str).str.s
    df["icao24"] = df["icao24"].str.replace(r"\.0$", "", regex=True)


for b in ["onground","alert","spi"]:
    if b in df.columns:
        df[b] = df[b].astype(str).str.strip().str.upper().map({"TRUE": 1, "FALSE": 0, "1": 1, "0": 0}).fillna(0).astype(int)
if "lat" in df.columns and "lon" in df.columns:
    df = df[df["lat"].between(-90, 90) & df["lon"].between(-180, 180)]
if "velocity" in df.columns:
    df.loc[df["velocity"] < 0, "velocity"] = np.nan
    df.loc[df["velocity"] > 500, "velocity"] = np.nan  # 500 m/s ~ 1800 km/h cap

if "time" not in df.columns:
    raise RuntimeError("Input lacks 'time' column")
df = df.sort_values(["icao24","time"]) if "icao24" in df.columns else df.sort_values("time")
df = df.reset_index(drop=True)

print("After initial cleaning shape:", df.shape)

print("Computing derived features...")

group_key = "icao24" if "icao24" in df.columns else None
derived_cols = []
if group_key:
    df["_dt"] = df.groupby(group_key)["time"].diff().fillna(0)
    df["_dlat"] = df.groupby(group_key)["lat"].diff().fillna(0)
    df["_dlon"] = df.groupby(group_key)["lon"].diff().fillna(0)
    df["_dalt"] = df.groupby(group_key)["baroaltitude"].diff().fillna(0)
    df["_dspeed"] = df.groupby(group_key)["velocity"].diff().fillna(0)
    df["_dvert"] = df.groupby(group_key)["vertrate"].diff().fillna(0)
else:
    df["_dt"] = df["time"].diff().fillna(0)
    df["_dlat"] = df["lat"].diff().fillna(0)
    df["_dlon"] = df["lon"].diff().fillna(0)
    df["_dalt"] = df["baroaltitude"].diff().fillna(0)
    df["_dspeed"] = df["velocity"].diff().fillna(0)
    df["_dvert"] = df["vertrate"].diff().fillna(0)

derived_cols += ["_dt","_dlat","_dlon","_dalt","_dspeed","_dvert"]

df["_haversine_m"] = haversine(df["lat"], df["lon"],
                               df["lat"].shift(1), df["lon"].shift(1)).fillna(0)
if group_key:

    mask_change = df[group_key] != df[group_key].shift(1)
    df.loc[mask_change, "_haversine_m"] = 0.0
derived_cols.append("_haversine_m")

df["_accel"] = df["_dspeed"] / df["_dt"].replace({0: np.nan})
df["_accel"] = df["_accel"].replace([np.inf, -np.inf], np.nan).fillna(0)
derived_cols.append("_accel")

df["_alt_rate"] = df["_dalt"] / df["_dt"].replace({0: np.nan})
df["_alt_rate"] = df["_alt_rate"].replace([np.inf, -np.inf], np.nan).fillna(0)
derived_cols.append("_alt_rate")

if "vertrate" in df.columns:
    df["_vert_abs"] = df["vertrate"].abs().fillna(0)
    derived_cols.append("_vert_abs")

if "heading" in df.columns:
   
    df["heading_filled"] = df["heading"].fillna(method="ffill").fillna(0)
    def cyc_diff(a, b):

        diff = (b - a + 180) % 360 - 180
        return diff
    df["_dheading"] = (df.groupby(group_key)["heading_filled"].diff().fillna(0)
                       .apply(lambda x: (x + 180) % 360 - 180))
    
    
    df["_turn_rate"] = df["_dheading"] / df["_dt"].replace({0: np.nan})
    df["_turn_rate"] = df["_turn_rate"].replace([np.inf, -np.inf], np.nan).fillna(0)
    derived_cols += ["_dheading", "_turn_rate"]
if "geoaltitude" in df.columns and "baroaltitude" in df.columns:
    df["_alt_diff_geo_baro"] = df["geoaltitude"] - df["baroaltitude"]
    derived_cols.append("_alt_diff_geo_baro")


df["_h_speed_mps"] = df["_haversine_m"] / df["_dt"].replace({0: np.nan})
df["_h_speed_mps"] = df["_h_speed_mps"].replace([np.inf, -np.inf], np.nan).fillna(0)
derived_cols.append("_h_speed_mps")


base_features = [c for c in ["velocity","vertrate","baroaltitude","geoaltitude","heading","lat","lon"] if c in df.columns]

feature_cols = base_features + derived_cols

feature_cols = [c for i,c in enumerate(feature_cols) if c in df.columns and feature_cols.index(c)==i]

print("Feature columns:", feature_cols)


essential = ["velocity", "baroaltitude", "lat", "lon"]
essential = [c for c in essential if c in df.columns]
df = df.dropna(subset=essential)
print("After dropping NA essentials shape:", df.shape)


X = df[feature_cols].fillna(0).values
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

with open(SCALER_FILE, "wb") as f:
    pickle.dump(scaler, f)
print("Saved scaler to", SCALER_FILE)


pca = PCA(n_components=min(10, Xs.shape[1]))
Xs_pca = pca.fit_transform(Xs)
with open(PCA_FILE, "wb") as f:
    pickle.dump(pca, f)
print("Saved PCA to", PCA_FILE)

for i in range(Xs_pca.shape[1]):
    df[f"pc{i+1}"] = Xs_pca[:, i]


print("Fitting IsolationForest (baseline)...")
if_model = IsolationForest(n_estimators=200, contamination=0.01, random_state=42, n_jobs=-1)
if_model.fit(Xs)
df["anomaly_score_if"] = -if_model.decision_function(Xs)  # higher = more anomalous

thr = float(np.quantile(df["anomaly_score_if"], 0.99))
df["is_anomaly_if"] = (df["anomaly_score_if"] >= thr).astype(int)

with open(IF_FILE, "wb") as f:
    pickle.dump(if_model, f)
print("Saved IsolationForest to", IF_FILE, " flagged:", int(df["is_anomaly_if"].sum()))


if TF_AVAILABLE:
    print("TensorFlow found — training autoencoder (this may take a while on CPU)...")
    input_dim = Xs.shape[1]
    latent_dim = max(4, min(32, input_dim // 2))

    def build_autoencoder(input_dim, latent_dim):
        inp = layers.Input(shape=(input_dim,))
        x = layers.Dense(128, activation="relu")(inp) if input_dim >= 8 else layers.Dense(64, activation="relu")(inp)
        x = layers.Dense(64, activation="relu")(x)
        encoded = layers.Dense(latent_dim, activation="relu")(x)
        x = layers.Dense(64, activation="relu")(encoded)
        x = layers.Dense(128, activation="relu")(x)
        out = layers.Dense(input_dim, activation="linear")(x)
        auto = models.Model(inputs=inp, outputs=out)
        auto.compile(optimizer="adam", loss="mse")
        return auto

    ae = build_autoencoder(input_dim, latent_dim)
    sample_idx = np.random.choice(len(Xs), size=min(100000, len(Xs)), replace=False)
    Xs_train = Xs[sample_idx]

    es = callbacks.EarlyStopping(monitor="loss", patience=5, restore_best_weights=True)
    hist = ae.fit(Xs_train, Xs_train,
                  epochs=50, batch_size=512, shuffle=True, callbacks=[es], verbose=1)

    rec = ae.predict(Xs, batch_size=2048)
    rec_err = np.mean((rec - Xs)**2, axis=1)
    df["anomaly_score_ae"] = rec_err
    ae_thr = float(np.quantile(rec_err, 0.99))
    df["is_anomaly_ae"] = (rec_err >= ae_thr).astype(int)

    
    df["anomaly_score_combined"] = np.maximum(df["anomaly_score_if"], df["anomaly_score_ae"])
    df["is_anomaly_combined"] = ((df["is_anomaly_if"] == 1) | (df["is_anomaly_ae"] == 1)).astype(int)

    ae.save(AE_FILE)
    with open(AE_HISTORY, "wb") as f:
        pickle.dump(hist.history, f)
    print("Saved autoencoder and history:", AE_FILE, AE_HISTORY)
else:
    print("TensorFlow not available. Skipping autoencoder. To enable AE install tensorflow.")
    df["anomaly_score_combined"] = df["anomaly_score_if"]
    df["is_anomaly_combined"] = df["is_anomaly_if"]


print("Saving cleaned features CSV to", CLEANED_CSV)
df.to_csv(CLEANED_CSV, index=False)

summary = {
    "n_rows": int(len(df)),
    "n_aircraft": int(df["icao24"].nunique()) if "icao24" in df.columns else "N/A",
    "n_flagged_if": int(df["is_anomaly_if"].sum()),
    "n_flagged_combined": int(df["is_anomaly_combined"].sum())
}
print("Summary:", summary)


if PLOTLY_AVAILABLE:
    try:
        
        fig1 = px.histogram(df, x="is_anomaly_if", title="IsolationForest anomaly counts", labels={"is_anomaly_if":"is_anomaly_if"})
        fig1.write_html(OUT_DIR / "if_anomaly_counts.html")
        if "pc1" in df.columns and "pc2" in df.columns:
            fig2 = px.scatter(df.sample(min(20000, len(df))), x="pc1", y="pc2",
                              color="is_anomaly_combined", title="PC1 vs PC2 (anomaly color)")
            fig2.write_html(OUT_DIR / "pc_scatter.html")
        print("Saved plotly html files in", OUT_DIR)
    except Exception as e:
        print("Plotly plots failed:", e)
else:
    print("Plotly not installed — skipping interactive plots.")

print("Pipeline complete. Artifacts in:", OUT_DIR)


Loading CSV: C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\adsb_sample.csv
Raw shape: (1526689, 16)
After initial cleaning shape: (1075733, 16)
Computing derived features...


C:\Users\Priyanga N\AppData\Local\Temp\ipykernel_27208\1344945169.py:171: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



Feature columns: ['velocity', 'vertrate', 'baroaltitude', 'geoaltitude', 'heading', 'lat', 'lon', '_dt', '_dlat', '_dlon', '_dalt', '_dspeed', '_dvert', '_haversine_m', '_accel', '_alt_rate', '_vert_abs', '_dheading', '_turn_rate', '_alt_diff_geo_baro', '_h_speed_mps']
After dropping NA essentials shape: (957295, 31)
Saved scaler to c:\Users\Priyanga N\AppData\Local\Programs\Microsoft VS Code\pipeline_artifacts\scaler.pkl
Saved PCA to c:\Users\Priyanga N\AppData\Local\Programs\Microsoft VS Code\pipeline_artifacts\pca.pkl
Fitting IsolationForest (baseline)...
Saved IsolationForest to c:\Users\Priyanga N\AppData\Local\Programs\Microsoft VS Code\pipeline_artifacts\isolation_forest.pkl  flagged: 9573
TensorFlow found — training autoencoder (this may take a while on CPU)...
Epoch 1/50
196/196 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.7415
Epoch 2/50
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.2128
Epoch 3/50
196/196 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.1195
Epoch 4/50
196/196 

Saved autoencoder and history: c:\Users\Priyanga N\AppData\Local\Programs\Microsoft VS Code\pipeline_artifacts\autoencoder.h5 c:\Users\Priyanga N\AppData\Local\Programs\Microsoft VS Code\pipeline_artifacts\ae_history.pkl
Saving cleaned features CSV to c:\Users\Priyanga N\AppData\Local\Programs\Microsoft VS Code\pipeline_artifacts\cleaned_features.csv
Summary: {'n_rows': 957295, 'n_aircraft': 6068, 'n_flagged_if': 9573, 'n_flagged_combined': 14856}
Saved plotly html files in c:\Users\Priyanga N\AppData\Local\Programs\Microsoft VS Code\pipeline_artifacts
Pipeline complete. Artifacts in: c:\Users\Priyanga N\AppData\Local\Programs\Microsoft VS Code\pipeline_artifacts


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import joblib

df = pd.read_csv(r"C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\cleaned_adsb.csv")

features = ["velocity","vertrate","baroaltitude","geoaltitude","lat","lon"]
df_clust = df[features].dropna()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clust)

kmeans = KMeans(n_clusters=6, random_state=42, n_init='auto')
kmeans.fit(X_scaled)

joblib.dump(kmeans, r"C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\kmeans_model.pkl")
print("Model saved!")

Model saved!


In [28]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

CLUSTERED_PATH = r"C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\clustered_adsb.csv"
MODEL_PATH = r"C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\kmeans_model.pkl"
ANOMALY_OUT = r"C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\adsb_anomalies.csv"

df = pd.read_csv(CLUSTERED_PATH)

# Features used for clustering
features = ["velocity", "vertrate", "baroaltitude", "geoaltitude", "lat", "lon"]

df_clean = df.dropna(subset=features).copy()

# Scale again (same approach used earlier)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clean[features])

# Load model
kmeans = joblib.load(MODEL_PATH)

# Distances to cluster centers
distances = np.min(
    np.linalg.norm(
        X_scaled[:, None, :] - kmeans.cluster_centers_[None, :, :], axis=2
    ),
    axis=1
)

df_clean["anomaly_score"] = distances

# Top 0.5% most anomalous
threshold = np.quantile(distances, 0.995)
anomalies = df_clean[df_clean["anomaly_score"] >= threshold]

anomalies.to_csv(ANOMALY_OUT, index=False)
print("Anomalies saved:", ANOMALY_OUT)
anomalies.head()


Anomalies saved: C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\adsb_anomalies.csv


,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact,phase,anomaly_score
6,1496620800,c04081,51.499400,7.581339,222.683105,310.127563,-293.37000,TFL230,0,0,0,1000.0,11414.76,11452.86,1.496621e+09,1.496621e+09,4.0,19.856306
8,1496620800,34310d,51.842058,12.528298,218.726807,37.353516,681.99000,FAH5102,0,0,0,3611.0,5433.06,6050.28,1.496621e+09,1.496621e+09,3.0,10.101886
18,1496620800,89629b,49.830036,10.567131,258.063721,101.266479,371.47500,ETD946,0,0,0,7675.0,10584.18,10736.58,1.496621e+09,1.496621e+09,3.0,11.019462
565,1496620810,adfbde,40.635041,-73.758899,0.000000,180.000000,-0.32512,TEST1234,1,0,0,7373.0,10972.80,11818.62,1.496621e+09,1.496621e+09,5.0,3.246675
577,1496620810,4846fd,50.371857,10.168497,95.249970,153.019769,-85.83168,REFT3BTM,0,1,1,2467.0,9044.94,33588.96,1.496611e+09,1.496621e+09,4.0,8.059723


In [29]:
import plotly.express as px

fig = px.scatter(
    anomalies,
    x="velocity",
    y="baroaltitude",
    color="anomaly_score",
    title="Anomalies: Velocity vs Altitude",
    color_continuous_scale="Turbo"
)
fig.show()


In [ ]:
if "time" in anomalies.columns:
    
    sample_n = min(30000, len(anomalies))  
    
    fig = px.scatter(
        anomalies.sort_values("time").sample(sample_n),
        x="time",
        y="velocity",
        color="anomaly_score",
        title="Velocity Over Time — Anomaly Highlight",
        color_continuous_scale="Viridis"
    )
    fig.show()


In [ ]:
df = pd.read_csv(r"C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\cleaned_adsb.csv")

df = df.sort_values(["icao24", "time"])

df["dt"] = df.groupby("icao24")["time"].diff()

print(df["dt"].describe())


count    1.516478e+06
mean     1.093678e+01
std      3.355150e+01
min      1.000000e+01
25%      1.000000e+01
50%      1.000000e+01
75%      1.000000e+01
max      3.220000e+03
Name: dt, dtype: float64


In [15]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\cleaned_adsb.csv")
df = df.sort_values(["icao24", "time"]).reset_index(drop=True)


top_icao = df["icao24"].value_counts().idxmax()
dff = df[df["icao24"] == top_icao].copy()

print("Using aircraft:", top_icao)
print(dff.shape)


Using aircraft: 484f2e
(360, 16)


In [23]:
dff["velocity"].describe()


count    351.000000
mean     195.429094
std       45.032430
min       67.976723
25%      199.666086
50%      218.048584
75%      220.648438
max      229.239258
Name: velocity, dtype: float64

In [ ]:

import os
import math
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import plotly.express as px
import plotly.graph_objects as go

DATA_PATH = r"C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\cleaned_adsb.csv"
OUT_DIR = r"C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection"
MODEL_OUT = os.path.join(OUT_DIR, "hgb_velocity_model.joblib")
usecols = ["time","icao24","lat","lon","velocity","vertrate","baroaltitude","geoaltitude","heading"]
df = pd.read_csv(DATA_PATH, usecols=usecols, low_memory=False)
df = df.sort_values(["icao24","time"]).reset_index(drop=True)
print("Loaded rows:", df.shape[0])

top_icao = df["icao24"].value_counts().idxmax()
dff = df[df["icao24"] == top_icao].copy().reset_index(drop=True)
print("Selected icao24:", top_icao, "rows:", len(dff))

num_cols = ["velocity","vertrate","baroaltitude","geoaltitude","lat","lon","heading"]
dff[num_cols] = dff[num_cols].replace([np.inf, -np.inf], np.nan).astype(float)

dff = dff.dropna(subset=["velocity"]).reset_index(drop=True)

if len(dff) < 200:
    raise RuntimeError(f"Not enough rows for {top_icao} (only {len(dff)}). Choose another aircraft or use more data.")
dff["ds"] = pd.to_datetime(dff["time"], unit="s")
dff["hour"] = dff["ds"].dt.hour
dff["minute"] = dff["ds"].dt.minute
dff["second"] = dff["ds"].dt.second

dff["hour_sin"] = np.sin(2 * np.pi * dff["hour"] / 24)
dff["hour_cos"] = np.cos(2 * np.pi * dff["hour"] / 24)
dff["min_sin"] = np.sin(2 * np.pi * dff["minute"] / 60)
dff["min_cos"] = np.cos(2 * np.pi * dff["minute"] / 60)

dff["dt"] = dff["time"].diff().fillna(0).astype(float)

def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1 = np.radians(lat1); phi2 = np.radians(lat2)
    dphi = np.radians(lat2 - lat1); dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2.0)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2.0)**2
    return R * 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

dff["lat_prev"] = dff["lat"].shift(1)
dff["lon_prev"] = dff["lon"].shift(1)
dff["dist_m"] = haversine_m(dff["lat_prev"].fillna(dff["lat"]), dff["lon_prev"].fillna(dff["lon"]),
                            dff["lat"], dff["lon"])
dff["speed_m_s"] = dff["dist_m"] / dff["dt"].replace(0, np.nan)  
dff["speed_m_s"] = dff["speed_m_s"].fillna(0)

LAGS = [1,2,3,5,10] 
for lag in LAGS:
    dff[f"vel_lag_{lag}"] = dff["velocity"].shift(lag)
    dff[f"vel_diff_{lag}"] = dff["velocity"] - dff[f"vel_lag_{lag}"]

ROLL_WINDOWS = [3,5,10]
for w in ROLL_WINDOWS:
    dff[f"vel_rollmean_{w}"] = dff["velocity"].rolling(window=w, min_periods=1).mean()
    dff[f"vel_rollstd_{w}"] = dff["velocity"].rolling(window=w, min_periods=1).std().fillna(0)

dff["alt_diff_1"] = dff["baroaltitude"] - dff["baroaltitude"].shift(1)
dff["geo_minus_baro"] = dff["geoaltitude"] - dff["baroaltitude"]

dff["heading_sin"] = np.sin(np.deg2rad(dff["heading"].fillna(0)))
dff["heading_cos"] = np.cos(np.deg2rad(dff["heading"].fillna(0)))

dff = dff.dropna(subset=[f"vel_lag_{max(LAGS)}"]).reset_index(drop=True)

median_dt = int(np.median(dff["dt"].replace(0, np.nan).dropna()))
print("median dt (s):", median_dt)
step_ahead = max(1, round(30 / max(1, median_dt)))  
print("step_ahead rows (≈30s):", step_ahead)

dff["y"] = dff["velocity"].shift(-step_ahead)  
dff = dff.dropna(subset=["y"]).reset_index(drop=True)

n = len(dff)
test_size = int(0.2 * n)
train = dff.iloc[: n - test_size].copy()
test  = dff.iloc[n - test_size :].copy()

print("Train rows:", len(train), "Test rows:", len(test))

features = [
    "velocity", "vel_lag_1", "vel_lag_2", "vel_lag_3", "vel_lag_5",
    "vel_diff_1", "vel_diff_2", "vel_diff_3", "vel_diff_5",
    "vel_rollmean_3", "vel_rollmean_5", "vel_rollmean_10",
    "vel_rollstd_3", "vel_rollstd_5",
    "vertrate", "baroaltitude", "geoaltitude", "geo_minus_baro",
    "speed_m_s",
    "hour_sin", "hour_cos", "min_sin", "min_cos",
    "heading_sin", "heading_cos",
    "alt_diff_1"
]
features = [f for f in features if f in train.columns]
print("Using features:", features)

X_train = train[features].fillna(0).astype(float)
y_train = train["y"].astype(float)
X_test  = test[features].fillna(0).astype(float)
y_test  = test["y"].astype(float)

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

model = HistGradientBoostingRegressor(
    max_iter=600,
    learning_rate=0.05,
    max_depth=10,
    random_state=42
)

print("Training model...")
model.fit(X_train_s, y_train)
print("Done training.")

from sklearn.metrics import mean_squared_error
import numpy as np

preds_test = model.predict(X_test_s)

mse = mean_squared_error(y_test, preds_test)
rmse = np.sqrt(mse)

print(f"Velocity Forecast RMSE: {rmse:.2f}")

import plotly.express as px
compare_df = pd.DataFrame({
    "time": X_test.index,
    "actual_velocity": y_test.values,
    "predicted_velocity": preds_test
})

fig = px.line(
    compare_df.sort_values("time"),
    x="time", 
    y=["actual_velocity", "predicted_velocity"],
    title="Actual vs Predicted Velocity"
)
fig.show()

joblib.dump(model, MODEL_OUT)
joblib.dump(scaler, SCALER_OUT)
print("Saved model ->", MODEL_OUT)
print("Saved scaler ->", SCALER_OUT)

plot_df = test[["ds"]].copy().reset_index(drop=True)
plot_df["actual"] = y_test.values
plot_df["predicted"] = preds_test

fig = go.Figure()
fig.add_trace(go.Scatter(x=plot_df["ds"], y=plot_df["actual"], mode="lines", name="Actual"))
fig.add_trace(go.Scatter(x=plot_df["ds"], y=plot_df["predicted"], mode="lines", name="Predicted"))
fig.update_layout(title=f"Velocity Forecast — {top_icao} (RMSE: {rmse:.1f})",
                  xaxis_title="time", yaxis_title="velocity")
fig.show()

try:
    importances = getattr(model, "feature_importances_", None)
    if importances is not None:
        fi = pd.Series(importances, index=features).sort_values(ascending=False)
        print("Top features:\n", fi.head(10))
except Exception:
    pass


Loaded rows: 1526689
Selected icao24: 484f2e rows: 360
median dt (s): 10
step_ahead rows (≈30s): 3
Train rows: 278 Test rows: 69
Using features: ['velocity', 'vel_lag_1', 'vel_lag_2', 'vel_lag_3', 'vel_lag_5', 'vel_diff_1', 'vel_diff_2', 'vel_diff_3', 'vel_diff_5', 'vel_rollmean_3', 'vel_rollmean_5', 'vel_rollmean_10', 'vel_rollstd_3', 'vel_rollstd_5', 'vertrate', 'baroaltitude', 'geoaltitude', 'geo_minus_baro', 'speed_m_s', 'hour_sin', 'hour_cos', 'min_sin', 'min_cos', 'heading_sin', 'heading_cos', 'alt_diff_1']
Training model...
Done training.
Velocity Forecast RMSE: 70.18


Saved model -> C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\hgb_velocity_model.joblib
Saved scaler -> C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\hgb_velocity_scaler.joblib


In [30]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\Priyanga N\OneDrive\Documents\flight-anomaly-detection\cleaned_adsb.csv")
df = df.sort_values(["icao24", "time"]).reset_index(drop=True)

# pick aircraft with max data
top_icao = df["icao24"].value_counts().idxmax()
dff = df[df["icao24"] == top_icao].copy()

print("Using aircraft:", top_icao)
print(dff.shape)

# remove missing values for velocity
dff = dff.dropna(subset=["velocity"]).reset_index(drop=True)

# lag features
dff["vel_lag1"] = dff["velocity"].shift(1)
dff["vel_lag2"] = dff["velocity"].shift(2)

# moving averages
dff["vel_ma5"] = dff["velocity"].rolling(5).mean()
dff["vel_ma10"] = dff["velocity"].rolling(10).mean()

# vertical rate smoothers
dff["vr_ma5"] = dff["vertrate"].rolling(5).mean()

# time features
dff["ds"] = pd.to_datetime(dff["time"], unit="s")
dff["hour"] = dff["ds"].dt.hour
dff["minute"] = dff["ds"].dt.minute
dff["sin_hour"] = np.sin(2*np.pi * dff["hour"] / 24)
dff["cos_hour"] = np.cos(2*np.pi * dff["hour"] / 24)

# drop rows created due to lag/rolling NaN
dff = dff.dropna().reset_index(drop=True)

print("Final rows:", dff.shape)

features = [
    "baroaltitude", "geoaltitude",
    "vertrate",
    "lat", "lon", "heading",
    "vel_lag1", "vel_lag2",
    "vel_ma5", "vel_ma10",
    "vr_ma5",
    "sin_hour", "cos_hour"
]

X = dff[features]
y = dff["velocity"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)
from xgboost import XGBRegressor

model = XGBRegressor(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=7,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    tree_method="hist"   # fast
)

model.fit(X_train_s, y_train)
from sklearn.metrics import mean_squared_error

preds = model.predict(X_test_s)
rmse = np.sqrt(mean_squared_error(y_test, preds))

print(f"Velocity Prediction RMSE: {rmse:.2f} m/s")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Priyanga N\\OneDrive\\Documents\\flight-anomaly-detection\\cleaned_adsb.csv'